# <p style="text-align:center">TP5 : Classification</p>
L’objectif de ce TP est de programmer et de tester deux algorithmes de classification, très
simples mais très efficaces : l’algorithme du **Plus Proche Voisin (PPV) (A.)** et **le Classifieur
Bayesien Naïf (CBN) (B.)** et **Arbre de decision (C.)** . Nous n’étudions ici que les versions les plus
simples de ces algorithmes. Pour ce TP nous aurons besoin d’importer sklearn et numpy. Les
tests pourront se faire sur les données prédéfinies de sklearn avec leurs étiquettes de classe
(target), par exemple :

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [5]:
# Jeu de donnee
iris = load_iris ()
X = iris.data
Y = iris.target

In [6]:
dfX = pd.DataFrame(X)
dfX.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [7]:
dfY = pd.DataFrame(Y)
dfY.head()

,0
0,0
1,0
2,0
3,0
4,0


# A. Plus Proche Voisin

In [8]:
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn import neighbors
from sklearn.metrics.pairwise import euclidean_distances

## 1.
Créez une fonction PPV(X,Y) qui prend en entrée des données X et des étiquettes Y et qui
renvoie une étiquette, pour chaque donnée, prédite à partir du plus proche voisin de cette
donnée. Ici on prend chaque donnée, une par une, comme donnée de test et on considère
toutes les autres comme données d’apprentissage. Cela nous permet de tester la puissance
de notre algorithme selon une méthode de validation par validation croisée (cross validation)
de type “leave one out”.

In [9]:
def PPV(x, y):
    """ PPV : Plus Proche Voisin
    Principe : la donnee de test est classe selon les K
    plus proches voisin de classe majoritaire.
    """
    # NOMBRE DE VOISIN
    k = 1
    # Taille de données
    taille = len(x)

    # ppv_target contient les nouvelles etiquettes: np.array([0, 0, ...])
    ppv_target = np.zeros(taille, dtype=np.int)

    # 1-folds Cross validation
    for i in range(taille):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)
        assert taille-1 == len(train)

        # calculer la distance entre valid et les autres données (train)
        dist = list(euclidean_distances(train, valid.reshape(1, -1)))

        # selectioner la classe de la plus proche donnee
        min_dist_idx = np.argmin(dist)
        ppv_target[i] = train_target[min_dist_idx]

    return {'target': ppv_target}


## 2.
La fonction PPV calcule une étiquette prédite pour chaque donnée. Modifiez la fonction
pour calculer et renvoyer l’erreur de prédiction : c’est à dire le pourcentage d’étiquettes
mal prédites.

In [10]:
def validateur_malprediction(lis_1, lis_2):
    """ validateur_malprediction : Retourne le pourcentage d’étiquettes mal prédites
    """
    size = len(lis_1)
    if size != len(lis_2):
        logging.error("Liste de taille differente")
        return 0
    # 1) trouver le total d’étiquettes mal prédites
    _sum = 0
    for i in range(size):
        if lis_1[i] != lis_2[i]:
            _sum += 1
    # 2) retourner le facteur en %
    return _sum*100 / float(size)

###### PPV(x,y) updated

In [11]:
def PPV(x, y):
    """ PPV : Plus Proche Voisin
    Principe : la donnee de test est classe selon les K
    plus proches voisin de classe majoritaire.
    """
    # NOMBRE DE VOISIN
    k = 1
    # Taille de données
    taille = len(x)

    # ppv_target contient les nouvelles etiquettes: np.array([0, 0, ...])
    ppv_target = np.zeros(taille, dtype=np.int)

    # Cross validation
    for i in range(taille):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)
        assert taille-1 == len(train)

        # calculer la distance entre valid et les autres données (train)
        dist = list(euclidean_distances(train, valid.reshape(1, -1)))

        # selectioner la classe de la plus proche donnee
        min_dist_idx = np.argmin(dist)
        ppv_target[i] = train_target[min_dist_idx]

    return {'target': ppv_target, 'error': validateur_malprediction(ppv_target, y) }


In [12]:
np.multiply([1,2], [2, 3])

array([2, 6])

## 3.
Testez sur les données Iris

## 4.
Testez la fonction des K Plus Proches Voisins de sklearn (avec ici K = 1). Les résultats
sont-ils différents ? Testez avec d’autres valeurs de K.

In [13]:
def predic_cross_valid(algo, x, y):
    """train : cross validation pour l'algorithme de sklearn
        x = data
        y = target
    """
    # taille de jeu de donnees
    taille = len(x)

    predicted_target = np.zeros(taille, dtype=np.int)

    for i in range(taille):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)
        
        # trouver la classe
        algo.fit(train, train_target)
        predicted_target[i] = algo.predict(valid.reshape(1, -1))

    return {'target': predicted_target, 'error': validateur_malprediction(predicted_target, y)}


In [14]:
ppv_target = PPV(x=X, y=Y)

target_sklearn = neighbors.KNeighborsClassifier(n_neighbors=1)

sklearn_ppv_target = predic_cross_valid(target_sklearn, X, Y)

def sep(n):
    print("_"*n)
    
print("Real : %s"%Y)
sep(100)
print("PPV : %s"% ppv_target['target'])
print("PPV pourcentage d’étiquettes mal prédites : %.1f"% ppv_target['error'])
sep(100)
print("KNeighbors : %s"% sklearn_ppv_target['target'])
print("KNeighbors pourcentage d’étiquettes mal prédites : %.1f"% sklearn_ppv_target['error'])

Real : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
____________________________________________________________________________________________________
PPV : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
PPV pourcentage d’étiquettes mal prédites : 4.0
____________________________________________________________________________________________________
KNeighbors : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 

Les résultats des fonction prédiction sont identiques mais différent de Y réel avec un taux de 4%!

#### AVEC ```k = 4```

In [15]:
k = 4
target_sklearn = neighbors.KNeighborsClassifier(n_neighbors=k)

sklearn_ppv_target = predic_cross_valid(target_sklearn, X, Y)

def sep(n):
    print("_"*n)
    
print("Real : %s"%Y)
sep(100)
print("PPV : %s"% ppv_target['target'])
print("PPV pourcentage d’étiquettes mal prédites : %.1f"% ppv_target['error'])
sep(100)
print("KNeighbors : %s"% sklearn_ppv_target['target'])
print("KNeighbors pourcentage d’étiquettes mal prédites : %.1f"% sklearn_ppv_target['error'])

Real : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
____________________________________________________________________________________________________
PPV : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
PPV pourcentage d’étiquettes mal prédites : 4.0
____________________________________________________________________________________________________
KNeighbors : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 

Les résultats des fonction prédiction sont identiques mais différent de Y réel avec un taux de 4%!

#### AVEC ```k = 8```

In [16]:
k = 8
target_sklearn = neighbors.KNeighborsClassifier(n_neighbors=k)

sklearn_ppv_target = predic_cross_valid(target_sklearn, X, Y)

def sep(n):
    print("_"*n)
    
print("Real : %s"%Y)
sep(100)
print("PPV : %s"% ppv_target['target'])
print("PPV pourcentage d’étiquettes mal prédites : %.1f"% ppv_target['error'])
sep(100)
print("KNeighbors : %s"% sklearn_ppv_target['target'])
print("KNeighbors pourcentage d’étiquettes mal prédites : %.1f"% sklearn_ppv_target['error'])

Real : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
____________________________________________________________________________________________________
PPV : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
PPV pourcentage d’étiquettes mal prédites : 4.0
____________________________________________________________________________________________________
KNeighbors : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 

Les résultats des fonction prédiction sont différents de Y réel avec un taux de 4% et 3.3%!

#### AVEC ```k = 12```

In [17]:
k = 12
target_sklearn = neighbors.KNeighborsClassifier(n_neighbors=k)

sklearn_ppv_target = predic_cross_valid(target_sklearn, X, Y)

def sep(n):
    print("_"*n)
    
print("Real : %s"%Y)
sep(100)
print("PPV : %s"% ppv_target['target'])
print("PPV pourcentage d’étiquettes mal prédites : %.1f"% ppv_target['error'])
sep(100)
print("KNeighbors : %s"% sklearn_ppv_target['target'])
print("KNeighbors pourcentage d’étiquettes mal prédites : %.1f"% sklearn_ppv_target['error'])

Real : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
____________________________________________________________________________________________________
PPV : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
PPV pourcentage d’étiquettes mal prédites : 4.0
____________________________________________________________________________________________________
KNeighbors : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 

Les résultats des fonction prédiction sont identiques mais différent de Y réel avec un taux de 4%!

## 5.
Modifiez la fonction PPV pour qu’elle prenne en entrée un nombre K de voisins (au
lieu de 1). La classe prédite sera alors la classe majoritaire parmi les K voisins.

In [18]:
def PPV(x, y, k=1):
    """ PPV : Plus Proche Voisin
    Principe : la donnee de test est classe selon les K
    plus proches voisin de classe majoritaire.
    """
    # Taille de données
    taille = len(x)

    # ppv_target contient les nouvelles etiquettes: np.array([0, 0, ...])
    ppv_target = np.zeros(taille, dtype=np.int)

    # Cross validation
    for i in range(taille):
        # leave one out
        valid = x[i]
        train = np.delete(x, i, 0)
        train_target = np.delete(y, i, 0)
        assert taille-1 == len(train)

        # calculer la distance entre valid et les autres données (train)
        dist = list(euclidean_distances(train, valid.reshape(1, -1)))

        # selectioner la classe de la plus proche donnee
        if k == 1: # un seul voisin (le plus proche)
            min_dist_idx = np.argmin(dist)
            ppv_target[i] = train_target[min_dist_idx]
        else: # k > 1, il faut trier
            # Tri croissant
            # recupérer les k premiers # garder les k premiers
            # recupérer les indices
            sorted_dist = np.sort(dist, kind='heapsort', axis=0)
            kmin_dist = sorted_dist[0:k]
            kmin_dist_index = [dist.index(e) for e in kmin_dist]

            # Compte le nombre d'elem par classe
            class_dist = Counter([train_target[e] for e in kmin_dist_index])

            # Recupérer la classe majoritaire
            _max = -1
            idx_max = -1
            for key, value in class_dist.items():
                if value > _max:
                    _max = value
                    idx_max = key

            ppv_target[i] = idx_max

    return {'target': ppv_target, 'error': validateur_malprediction(ppv_target, y)}


# <p style="text-align:center">Test</p>

In [19]:
k = 12
ppv_target = PPV(x=X, y=Y, k=k)

target_sklearn = neighbors.KNeighborsClassifier(n_neighbors=k)

sklearn_ppv_target = predic_cross_valid(target_sklearn, X, Y)

def sep(n):
    print("_"*n)
    
print("Real : %s"%Y)
sep(100)
print("PPV : %s"% ppv_target['target'])
print("PPV pourcentage d’étiquettes mal prédites : %.1f"% ppv_target['error'])
sep(100)
print("KNeighbors : %s"% sklearn_ppv_target['target'])
print("KNeighbors pourcentage d’étiquettes mal prédites : %.1f"% sklearn_ppv_target['error'])


Real : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
____________________________________________________________________________________________________
PPV : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 2]
PPV pourcentage d’étiquettes mal prédites : 4.0
____________________________________________________________________________________________________
KNeighbors : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 

Les résultats des fonction prédiction sont identiques mais différent de Y réel avec un taux de 4%!